In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import struct


class MnistData:
    def __init__(self, train_image_path, train_label_path,
                 test_image_path, test_label_path):
        # file path
        self.train_image_path = train_image_path
        self.train_label_path = train_label_path
        self.test_image_path = test_image_path
        self.test_label_path = test_label_path

        # get_data()，0:train data，1:test data
        self.train_images, self.train_labels = self.get_data(0)
        self.test_images, self.test_labels = self.get_data(1)

    def get_data(self, data_type):
        if data_type == 0:  
            image_path = self.train_image_path
            label_path = self.train_label_path
        else:  
            image_path = self.test_image_path
            label_path = self.test_label_path

        with open(image_path, 'rb') as file1:
            image_file = file1.read()
        with open(label_path, 'rb') as file2:
            label_file = file2.read()

        label_index = 0
        image_index = 0
        labels = []
        images = []
        images_rot90 = []
        images_rot180 = []
        images_rot270 = []
        images_rv1 = []
        images_rv2 = []

        # read file imformation
        magic, num_of_datasets, rows, columns = struct.unpack_from('>IIII',
                                                                   image_file, image_index)
        image_index += struct.calcsize('>IIII')

        for i in range(num_of_datasets):
            # read 784 unsigned byte
            temp = struct.unpack_from('>784B', image_file, image_index)
            temp = np.reshape(temp, (28, 28))
            temp = temp / 255
            images.append(temp)
            image_index += struct.calcsize('>784B') 

        # jump the describe imformation
        label_index += struct.calcsize('>II')
        labels = struct.unpack_from('>' + str(num_of_datasets) + 'B', label_file, label_index)

        # one-hot
        labels = np.eye(10)[np.array(labels)]

        return np.array(images), labels


In [ ]:
# Fashion-MINIST dataset
train_image_path = '/content/drive/My Drive/fasion_data/train-images-idx3-ubyte'
train_label_path = '/content/drive/My Drive/fasion_data/train-labels-idx1-ubyte'
test_image_path = '/content/drive/My Drive/fasion_data/t10k-images-idx3-ubyte'
test_label_path = '/content/drive/My Drive/fasion_data/t10k-labels-idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path)

print(type(data.train_images))
print(type(data.train_labels))

In [4]:
# MINIST dataset
train_image_path = '/content/drive/My Drive/data/train-images.idx3-ubyte'
train_label_path = '/content/drive/My Drive/data/train-labels.idx1-ubyte'
test_image_path = '/content/drive/My Drive/data/t10k-images.idx3-ubyte'
test_label_path = '/content/drive/My Drive/data/t10k-labels.idx1-ubyte'

data = MnistData(train_image_path, train_label_path,
                 test_image_path, test_label_path)

print(type(data.train_images))
print(type(data.train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:

print(type(data.train_images))
print(type(data.train_labels))

print(type(data.train_images[0]))
print(type(data.train_labels[0]))

print(data.train_images[0])
print(data.train_labels[0])

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
# Define model
def get_model():
    model = tf.keras.Sequential()

    model.add(layers.Flatten(input_shape=(28, 28)))

    # First fully connected layer
    model.add(layers.Dense(512, activation = 'relu'))
    model.add(layers.Dropout(0.2))

    # Softmax layer
    model.add(layers.Dense(10, activation = 'softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    
    # print model
    model.summary()

    return model

model = get_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(data.train_images, data.train_labels, epochs=20)

Epoch 1/20
1875/1875 [==============================] - 7s 2ms/step - loss: 0.3641 - accuracy: 0.8917
Epoch 2/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1014 - accuracy: 0.9691
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0690 - accuracy: 0.9782
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0536 - accuracy: 0.9832
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0394 - accuracy: 0.9873
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0344 - accuracy: 0.9888
Epoch 7/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0258 - accuracy: 0.9916
Epoch 8/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0249 - accuracy: 0.9914
Epoch 9/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0227 - accuracy: 0.9918
Epoch 10/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.020

In [ ]:
model.evaluate(data.test_images, data.test_labels)